In [1]:
import torch
import pandas as pd
import numpy as np


In [2]:
ratings = pd.read_csv('data/ratings.dat', sep='::', engine = 'python', header=None)
ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
movies = pd.read_csv('data/movies.dat', sep='::', engine = 'python',
                     encoding="ISO-8859-1", header = None)
movies.columns = ['MovieID', 'Title', 'Genres']

In [3]:
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [4]:
item_feature_matrix = ratings.pivot_table(index='UserID', columns='MovieID', values='Rating')
item_feature_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
mean_ratings = np.array(item_feature_matrix.mean(axis=1, skipna=True)).reshape(-1, 1)
mean_ratings[0:10]

array([[4.18867925],
       [3.71317829],
       [3.90196078],
       [4.19047619],
       [3.14646465],
       [3.90140845],
       [4.32258065],
       [3.88489209],
       [3.73584906],
       [4.11471322]])

In [6]:
item_feature_matrix.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
normalized_ratings = item_feature_matrix - mean_ratings
# if the rating is negative, set it to 0
# normalized_ratings = normalized_ratings.clip(lower=0)
normalized_ratings.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.811321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,-1.146465,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
rating_0 = normalized_ratings.fillna(0)

# take the first 100 usrs and 500 movies
r_n_matrix = rating_0.iloc[:100, :500]

rating_binary = (r_n_matrix != 0).astype(int)
common_ratings_count = np.array(rating_binary.T.dot(rating_binary)) # fix 1
print(common_ratings_count)

# find users who have rated both movie 0
user_0 = rating_binary[rating_binary.iloc[:, 0] == 1].index - 1 # index starts from 1, fix 2
print(user_0)

# find users who have rated both movie 2
user_2 = rating_binary[rating_binary.iloc[:, 2] == 1].index - 1 # index starts from 1
print(user_2)

# # print user 25's rating for movie 0
# print(r_n_matrix.iloc[25, 0])

# # print user 44's rating for movie 0
# print(r_n_matrix.iloc[44, 0])

# # print user 25's rating for movie 2
# print(r_n_matrix.iloc[25, 2])

# # print user 44's rating for movie 2
# print(r_n_matrix.iloc[44, 2])

# find common users between user_0 and user_2
common_users = np.intersect1d(user_0, user_2) # fix 3
print(common_users)

# find ratings of common users for movie 0
movie_0_ratings = r_n_matrix.iloc[common_users, 0].values # fix 4
print(movie_0_ratings)

# find ratings of common users for movie 2
movie_2_ratings = r_n_matrix.iloc[common_users, 2].values # fix 5
print(movie_2_ratings)

# calculate the cosine similarity between movie 0 and movie 2
product = np.dot(movie_0_ratings, movie_2_ratings)
norm_0 = np.linalg.norm(movie_0_ratings)
norm_2 = np.linalg.norm(movie_2_ratings)

score = product / (norm_0 * norm_2)

print(score)
print(0.5 + 0.5 * score)


[[32  8  2 ...  2  0  2]
 [ 8 14  1 ...  0  0  3]
 [ 2  1  3 ...  1  0  0]
 ...
 [ 2  0  1 ...  3  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 2  3  0 ...  0  0  3]]
Index([ 0,  5,  7,  8,  9, 17, 18, 20, 22, 25, 27, 33, 35, 37, 43, 44, 47, 48,
       50, 55, 59, 64, 67, 72, 74, 75, 77, 79, 89, 91, 95, 98],
      dtype='int64', name='UserID')
Index([25, 44, 61], dtype='int64', name='UserID')
[25 44]
[0.04       1.05387205]
[-0.96       -0.94612795]
-0.7284506645698782
0.13577466771506091


In [44]:

def compute_similarity_matrix_v2(rating, top_n=None, n_jobs=-1):
    
    min_common_users = 3
    rating_0 = rating.fillna(0)
    
    # Compute the count of common ratings for each movie pair
    rating_binary = (rating_0 != 0).astype(int)
    common_ratings_count = np.array(rating_binary.T.dot(rating_binary))

    # Initialize similarity matrix
    similarity_matrix = np.full((rating_0.shape[1], rating_0.shape[1]), np.nan)

    # Iterate over each pair of movies
    for i in range(rating_0.shape[1]):
        for j in range(i + 1, rating_0.shape[1]):
            # Check if the pair has enough common ratings
            if common_ratings_count[i, j] >= min_common_users:
                # Filter ratings to include only common users
                user_0 = rating_binary[rating_binary.iloc[:, i] == 1].index - 1 # get user index, not user id
                user_1 = rating_binary[rating_binary.iloc[:, j] == 1].index - 1 # get user index, not user id
                common_users = np.intersect1d(user_0, user_1) 
        
                # get movie i and move j ratings from common users
                ratings_i = rating_0.iloc[common_users, i].values
                ratings_j = rating_0.iloc[common_users, j].values

                # Compute dot product and norms for the filtered ratings
                dot_product = np.dot(ratings_i, ratings_j)
                norm_i = np.linalg.norm(ratings_i)
                norm_j = np.linalg.norm(ratings_j)

                # Compute similarity
                if norm_i > 0 and norm_j > 0:
                    similarity = 0.5 + 0.5 * (dot_product / (norm_i * norm_j))
                    similarity_matrix[i, j] = similarity
                    similarity_matrix[j, i] = similarity
                else:
                    print('One of the norms is 0 for i = {}, j = {}'.format(i, j))
                    similarity_matrix[i, j] = 0
                    similarity_matrix[j, i] = 0

    # Keep only the top N similarities for each item
    # if top_n is not None:
    #     # [Code to keep top N similarities]
    #     return similarity_matrix
    
    return similarity_matrix

specified_movies = [1, 10, 100, 1510, 260, 3212]
r_n_matrix = normalized_ratings.iloc[:100, :500]
reduced_similarity_matrix = compute_similarity_matrix_v2(r_n_matrix)
print(reduced_similarity_matrix.shape)
print(reduced_similarity_matrix)

(500, 500)
[[       nan 0.48217456        nan ...        nan        nan        nan]
 [0.48217456        nan        nan ...        nan        nan 0.00946393]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan 0.00946393        nan ...        nan        nan        nan]]


In [41]:
similarity_matrix = compute_similarity_matrix_v2(normalized_ratings)
print(similarity_matrix.shape)
similarity_matrix.loc[specified_movies, specified_movies].round(7)

KeyboardInterrupt: 

In [42]:
import numpy as np
from joblib import Parallel, delayed

def compute_similarity_for_pair(i, j, rating_0, rating_binary, common_ratings_count, min_common_users=3):
    if common_ratings_count[i, j] >= min_common_users:
        user_0 = rating_binary[rating_binary.iloc[:, i] == 1].index - 1
        user_1 = rating_binary[rating_binary.iloc[:, j] == 1].index - 1
        common_users = np.intersect1d(user_0, user_1)

        ratings_i = rating_0.iloc[common_users, i].values
        ratings_j = rating_0.iloc[common_users, j].values

        dot_product = np.dot(ratings_i, ratings_j)
        norm_i = np.linalg.norm(ratings_i)
        norm_j = np.linalg.norm(ratings_j)

        if norm_i > 0 and norm_j > 0:
            similarity = 0.5 + 0.5 * (dot_product / (norm_i * norm_j))
        else:
            similarity = 0

        return i, j, similarity
    else:
        return i, j, np.nan

def compute_similarity_matrix_parallel(rating, top_n=None, n_jobs=-1):
    min_common_users = 3
    rating_0 = rating.fillna(0)
    rating_binary = (rating_0 != 0).astype(int)
    common_ratings_count = np.array(rating_binary.T.dot(rating_binary))

    similarity_matrix = np.full((rating_0.shape[1], rating_0.shape[1]), np.nan)

    # Prepare list of movie pairs for parallel processing
    movie_pairs = [(i, j) for i in range(rating_0.shape[1]) for j in range(i + 1, rating_0.shape[1])]

    # Parallel computation
    results = Parallel(n_jobs=n_jobs)(delayed(compute_similarity_for_pair)(i, j, rating_0, rating_binary, common_ratings_count, min_common_users) for i, j in movie_pairs)

    # Update similarity matrix with results
    for i, j, similarity in results:
        similarity_matrix[i, j] = similarity
        similarity_matrix[j, i] = similarity

    # [Code to keep top N similarities, if needed]

    return similarity_matrix


In [58]:
r_n_matrix = normalized_ratings.iloc[:100, :500]
reduced_similarity_matrix = compute_similarity_matrix_parallel(r_n_matrix)
print(reduced_similarity_matrix.shape)
print(reduced_similarity_matrix)

similarity_matrix = compute_similarity_matrix_parallel(normalized_ratings)
print(similarity_matrix.shape)
similarity_matrix.loc[specified_movies, specified_movies].round(7)

(500, 500)
[[       nan 0.48217456        nan ...        nan        nan        nan]
 [0.48217456        nan        nan ...        nan        nan 0.00946393]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan 0.00946393        nan ...        nan        nan        nan]]


In [56]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed

def compute_similarity_matrix(rating, top_n=None, n_jobs=-1):
    """
    Compute the item-item similarity matrix for a given matrix of centered ratings using sparse matrices and parallel computation.

    Parameters:
    - centered_rating_matrix: pd.DataFrame, a DataFrame where rows represent movies,
      columns represent users, and values represent centered ratings.
    - top_n: int, the number of most similar items to keep for each item.
    - n_jobs: int, the number of jobs to run in parallel. -1 means using all processors.

    Returns:
    - pd.DataFrame or scipy.sparse matrix, the similarity matrix with movies as both rows and columns, containing top N similarities.
    """
    # for each movie, if user has rated it, set it to 1, otherwise set it to 0
    rating = rating.fillna(0)
    binary_rating = pd.DataFrame(np.where(rating > 0, 1, 0), index=rating.index, columns=rating.columns)
    import torch
    binary_tensor = torch.tensor(binary_rating.values, dtype=torch.float32)
    #binary_tensor = binary_tensor.cuda()

    rating_count = binary_tensor.matmul(binary_tensor.t())
    rating_count = rating_count.cpu().numpy()

    rating_count = np.where(rating_count < 3, 0, 1)
    cosine_sim_df = pd.DataFrame(index=rating.index, columns=rating.index)

    def compute_similarity(i, j, rating_matrix, rating_count):
        if rating_count[i, j]:
            vec_i = rating_matrix.iloc[i]
            vec_j = rating_matrix.iloc[j]
            similarity = np.dot(vec_i, vec_j) / (np.linalg.norm(vec_i) * np.linalg.norm(vec_j))
        else:
            similarity = 0
        return i, j, similarity

    # Parallel computation of cosine similarity
    results = Parallel(n_jobs=-1)(delayed(compute_similarity)(i, j, rating, rating_count)
                                  for i in range(len(rating))
                                  for j in range(i + 1, len(rating)))

    # Fill the DataFrame with the computed similarities
    for i, j, similarity in results:
        cosine_sim_df.iloc[i, j] = similarity
        cosine_sim_df.iloc[j, i] = similarity  # symmetry
    if top_n is None:
        return 0.5 + 0.5 * cosine_sim_df.replace(0, np.nan)

In [57]:
r_n_matrix = normalized_ratings.iloc[:100, :500]
reduced_similarity_matrix = compute_similarity_matrix(r_n_matrix.T)
print(reduced_similarity_matrix.shape)
print(reduced_similarity_matrix)

(500, 500)
MovieID       1         2    3    4    5         6    7    8    9        10   \
MovieID                                                                        
1             NaN  0.494087  NaN  NaN  NaN  0.475357  NaN  NaN  NaN  0.64797   
2        0.494087       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
3             NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
4             NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
5             NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
...           ...       ...  ...  ...  ...       ...  ...  ...  ...      ...   
509           NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
510           NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
511           NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
512           NaN       NaN  NaN  NaN  NaN       NaN  NaN  NaN  NaN      NaN   
513           NaN       NaN  

In [ ]:
# get the movie titles by movie ids
def get_movie_titles(movie_ids):
    return movies[movies['MovieID'].isin(movie_ids)]

In [102]:
def recommend_movies(user_rating, s_matrix, n_recommendations = 10):
    """
    Generate movie recommendations based on new user ratings and a sparse similarity matrix.

    Parameters:
    - user_ratings: np.array, user's ratings for movies; 0 indicates the movie hasn't been rated.
    - s_matrix: item-item similarity matrix.
    - n_recommendations: int, the number of recommendations to return.

    Returns:
    - List of movie indices representing the top N recommendations.
    """

    # number of movies
    n_movies = len(user_rating)

    # iterate through each movie and calculate the weighted rating for user
    for l in range(n_movies):
        if user_rating[l] != 0:
            # get the similarity score for moviel l
            l_s_scores = np.nan_to_num(s_matrix.iloc[l])
            
            # get the weighted sum for movie l
            weighted_sum = np.sum(user_rating * l_s_scores)

            # get the normalization factor
            norm_factor = np.sum(l_s_scores * (user_rating != 0))

            # get the weighted average for movie l
            weighted_avg = np.sum(weighted_sum) / norm_factor if norm_factor != 0 else 0
            #print(l, " ", weighted_sum, " ", norm_factor, " ", weighted_avg)
            user_rating[l] = weighted_avg

    # get the top N recommendations
    top_n = np.argsort(user_rating)[-n_recommendations:]

    return top_n

In [103]:
# generate a vector 1 x 3706, which represents the rating of user 1 to all movies
user_1 = ratings[ratings['UserID'] == 1181]
user_1_ratings = user_1.set_index('MovieID')['Rating']
user_1_ratings = user_1_ratings.reindex(range(1, 3707), fill_value=0)

movie_ids = recommend_movies(user_1_ratings.values, similarity_matrix, 10)

ValueError: operands could not be broadcast together with shapes (3706,) (6040,) 

In [75]:
# get the movie titles by movie ids
def get_movie_titles(movie_ids):
    return movies[movies['MovieID'].isin(movie_ids)]

get_movie_titles(movie_ids)

,MovieID,Title,Genres
536,540,Sliver (1993),Thriller
867,878,Bye-Bye (1995),Drama
910,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
1568,1610,"Hunt for Red October, The (1990)",Action|Thriller
1571,1613,Star Maps (1997),Drama
1573,1615,"Edge, The (1997)",Adventure|Thriller
1576,1619,Seven Years in Tibet (1997),Drama|War
2252,2321,Pleasantville (1998),Comedy
2782,2851,Saturn 3 (1979),Adventure|Sci-Fi|Thriller
